## Load Data

In [1]:
import numpy as np
from collections import Counter

In [2]:
path = "/Users/zefeng-zhang/data/aclImdb/"
train_path = path + 'train/'
test_path = path + 'test/'

In [3]:
data_raw_pos = sc.textFile(train_path + "pos/*.txt")
data_raw_neg = sc.textFile(train_path + "neg/*.txt")

In [4]:
data_raw_pos.first()

u'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [5]:
data_raw_pos = data_raw_pos.sample(False, 0.01, 1)
data_raw_neg = data_raw_neg.sample(False, 0.01, 1)

In [6]:
data_raw_pos.getNumPartitions()

12500

In [7]:
num_partitions = 8
data_raw_pos = data_raw_pos.repartition(num_partitions)
data_raw_neg = data_raw_neg.repartition(num_partitions)

## Training NB

In [8]:
# split into words
data_pos = data_raw_pos.flatMap(lambda x : x.split()).map(lambda x : (x, 1)).reduceByKey(lambda x, y : x+y)
data_pos.take(10)

[(u'government,', 1),
 (u'detective', 7),
 (u'astonishing', 1),
 (u'Day.', 2),
 (u'pound,', 1),
 (u'ridden', 1),
 (u'timely', 1),
 (u'foremost,', 1),
 (u'public.', 1),
 (u'sympathetic', 2)]

In [9]:
data_neg = data_raw_neg.flatMap(lambda x : x.split()).map(lambda x : (x, 1)).reduceByKey(lambda x, y : x+y)
data_neg.take(10)

[(u'/>of', 1),
 (u'aided', 1),
 (u'Suspects"ish', 1),
 (u'days.', 2),
 (u'dubbed', 1),
 (u'sympathetic', 1),
 (u'...except', 1),
 (u'known', 2),
 (u'helped', 2),
 (u'musicals.', 1)]

In [14]:
count_pos = data_pos.map(lambda x : x[1]).reduce(lambda x, y : x+y)
count_neg = data_neg.map(lambda x : x[1]).reduce(lambda x, y : x+y)
print count_pos, count_neg

29040 26850


In [15]:
v1 = data_pos.map(lambda x : x[0])
v2 = data_neg.map(lambda x : x[0])
v0 = v1.union(v2).distinct()
v = v0.count()
print v

12606


In [16]:
pos_denom = float(count_pos + v + 1)
neg_denom = float(count_neg + v + 1)

In [17]:
pos_prob = data_pos.map(lambda x : (x[0], np.log(float(x[1] + 1)/pos_denom)))
neg_prob = data_neg.map(lambda x : (x[0], np.log(float(x[1] + 1)/neg_denom)))

In [18]:
pos_prob.take(10)
neg_prob.take(10)

[(u'/>of', -9.8898195697707081),
 (u'aided', -9.8898195697707081),
 (u'Suspects"ish', -9.8898195697707081),
 (u'days.', -9.4843544616625426),
 (u'dubbed', -9.8898195697707081),
 (u'sympathetic', -9.8898195697707081),
 (u'...except', -9.8898195697707081),
 (u'known', -9.4843544616625426),
 (u'helped', -9.4843544616625426),
 (u'musicals.', -9.8898195697707081)]

In [19]:
pos_prob = dict(pos_prob.collect())
neg_prob = dict(neg_prob.collect())

In [20]:
pos_prob_b = sc.broadcast(pos_prob)
neg_prob_b = sc.broadcast(neg_prob)

## Prediction

In [21]:
def pred_class(doc):
    words = doc.split(" ")
    counts = Counter(words)
    log_pos = 0.0
    log_neg = 0.0
    for w in counts:
        log_pos += counts[w] * pos_prob_b.value.get(w, np.log(1.0/pos_denom)) # default probability of "unk"
        log_neg += counts[w] * neg_prob_b.value.get(w, np.log(1.0/neg_denom))
    if log_pos > log_neg:
        return "pos"
    return "neg"

In [24]:
test_raw_pos = sc.textFile(test_path + "pos/*.txt")
print test_raw_pos.first()
print test_raw_pos.take(2)
test_raw_neg = sc.textFile(test_path + "neg/*.txt")

I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.
[u"I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I

In [25]:
test_pos_res = test_raw_pos.map(pred_class)
test_pos_res.take(10)
pos_results = Counter(test_pos_res.collect())

test_neg_res = test_raw_neg.map(pred_class)
test_neg_res.take(10)
neg_results = Counter(test_neg_res.collect())

In [26]:
accuracy = (pos_results['pos'] + neg_results['neg'])/float(pos_results['pos'] + pos_results['neg'] + neg_results['pos'] + neg_results['neg'])
print accuracy

0.73064
